In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.utils import image_dataset_from_directory
from tensorflow import keras
from keras import layers
from keras.activations import swish
from keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

2024-05-22 17:41:30.440064: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 17:41:30.440192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 17:41:30.555359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
input_dir = "/kaggle/input/cifar-10-fake-image-dataset/cifar-10-dataset/" #! Kaggle path
working_dir = "/kaggle/working/" #! Kaggle path

In [3]:
NAME = "cifakeCNN{}".format(time.strftime("%Y%m%d-%H%M%S"))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
RUNNING_WITH_LIMITED_GPU = False
RUNNING_WITH_LIMITED_CPU = False
RESTORE_KAGGLE_DIRECTORY_FROM_DATASET = False

Num GPUs Available:  2


In [5]:
if RUNNING_WITH_LIMITED_GPU:
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      # """Restrict TensorFlow to only allocate 1GB of memory on the first GPU"""
      try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

In [6]:
if RUNNING_WITH_LIMITED_CPU:
    #! """When running locally, limits the threads to 1 (only uses one thread of one CPU core)"""
    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

In [7]:
if RESTORE_KAGGLE_DIRECTORY_FROM_DATASET:
    #! """When running on Kaggle, copies the input directory cifake_hyperband to working directory for
    #! tuner continuity/cumulative session training(loading backups from BackupAndRestore callback)."""
    from distutils.dir_util import copy_tree
    source_dir = "/kaggle/input/cifake_hyperband/"
    destination_dir = "/kaggle/working/"
    copy_tree(source_dir, destination_dir)

In [8]:
ds_train = image_dataset_from_directory(
    f'{input_dir}train',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
#     seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 100000 files belonging to 2 classes.


In [9]:
ds_test = image_dataset_from_directory(
    f'{input_dir}test',
    labels='inferred',
    label_mode='binary',
    image_size=[32,32],
    interpolation='nearest',
    batch_size=32,
    shuffle=True,
#     seed=69,
    #validation_split=None,
    subset=None,
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 20000 files belonging to 2 classes.


In [10]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [11]:
ds_train = (
    ds_train
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


In [12]:
ds_valid = (
    ds_test
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [13]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=14,
    min_delta=0.0005,
    restore_best_weights=True)

In [14]:
from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=12, min_lr=0.0008)

In [15]:
from keras.callbacks import ModelCheckpoint
timestamp = time.strftime("%Y%m%d-%H%M%S")
model_checkpoint = ModelCheckpoint(
    f'{working_dir}/best/best_model_{timestamp}.keras', monitor='val_loss', save_best_only=True, mode='min'
)

In [16]:
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    else:
        return lr * 0.97 #if lr > 0.001 else lr * 3

schedule_lr = LearningRateScheduler(scheduler)

In [17]:
backup_dir = f"{working_dir}/backup/latest"
backup_callback = keras.callbacks.BackupAndRestore(
    backup_dir, save_freq="epoch", delete_checkpoint=True
)

In [ ]:
!pip install keras-tuner
import keras_tuner
from keras_tuner import RandomSearch, Hyperband
from keras.models import Model
from keras.metrics import Accuracy, Precision, Recall, FBetaScore
from kerastuner import Objective

In [ ]:
modelAcc = keras.models.load_model('/kaggle/input/models/BESTcifakeCNN20240320-123957.keras', compile=False)
modelPre = keras.models.load_model('/kaggle/input/models/0BESTcifakeCNN20240513-003312 (1).keras', compile=False)

# Rename the models
modelAcc._name = 'model1'
modelPre._name = 'model2'

# Set both models to non-trainable
modelAcc.trainable = False 
modelPre.trainable = False

class WeightedAverageLayer(tf.keras.layers.Layer):
    def __init__(self, w1, w2, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1]

In [ ]:
def build_model(hp=None):
    models = [modelAcc, modelPre]

    model_input = Input(shape=(32, 32, 3))
    
    # Generate output from each model using the shared input
    model_outputs = [model(m) for model in models]
    
    ensemble_output = WeightedAverageLayer(hp.Float('WEIGHT_ACC', min_value=0.0, max_value=1), 1 - hp.Float('WEIGHT_ACC', min_value=0.0, max_value=1))(model_outputs)
    
    ensemble_model = Model(inputs=model_input, outputs=ensemble_output)
    
    ensemble_model.compile(optimizer=Adam(),
                           loss='binary_crossentropy',
                           metrics=['binary_accuracy',
                                    Precision(name='precision'),
                                    Recall(name='recall'),
                                    FBetaScore(beta=0.25)])
    
    return ensemble_model

In [22]:
weighted_obj = Objective('val_fbeta_score', direction='max')

In [23]:

tuner = Hyperband(
    build_model,
    objective=weighted_obj,
    max_epochs=5,
    factor=3,
    directory=f'{working_dir}/projects',
    project_name='cifake_hyperband')

In [ ]:
tuner.search(ds_train, epochs=40, validation_data=ds_valid, callbacks=[early_stopping,  reduce_lr, schedule_lr]) #, backup_callback,tensorboard, verbose=0 model_checkpoint
best_models = tuner.get_best_models(num_models=10)
for i in range(10):
    best_models[i].save(f"{working_dir}/{i}BEST{NAME}.keras")
best_model = best_models[0]

Trial 45 Complete [00h 03m 30s]
val_fbeta_score: 0.7444366216659546

Best val_fbeta_score So Far: 0.7444366216659546
Total elapsed time: 01h 32m 02s

Search: Running Trial #46

Value             |Best Value So Far |Hyperparameter
False             |False             |PRECONV_norm
44                |20                |out_channels_2
6                 |6                 |expansion_2
2                 |2                 |squeezing
False             |True              |use_se_2
0.1               |0.2               |conv6_dropout_rate_2
3                 |1                 |num_conv_layers
96                |32                |conv_0_filter
3                 |3                 |conv_kernel
False             |False             |l2_reg
0.1               |0                 |CONV_dropout_0_rate
True              |False             |CONV_0_pooling
True              |True              |CONV_0_norm
0                 |0.1               |dropout_INBCONV_rate
32                |96                |con

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

In [ ]:
best_model.save(f"{working_dir}/BEST{NAME}.keras")

In [ ]:
for i in range(10):
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Output the best model's metrics
    best_metrics = best_models[i].evaluate(ds_valid)
    print(f"\nModel Number {i}")
    print("\n")
    print("Validation Loss:", best_metrics[0])
    print("Validation Binary Accuracy:", best_metrics[1])
    print("Validation Precision:", best_metrics[2])
    print("Validation Recall:", best_metrics[3])
    print("Validation F1 Score:", best_metrics[4])